## Homework: Introduction

In this homework, we'll learn more about search and use Elastic Search for practice. 

> It's possible that your answers won't match exactly. If it's the case, select the closest one.

## Q1. Running Elastic 

Run Elastic Search 8.4.3, and get the cluster information. If you run it on localhost, this is how you do it:

```bash
curl localhost:9200
```

What's the `version.build_hash` value?


Running ElasticSearch:

```bash
docker run -it \
    --rm \
    --name elasticsearch \
    -p 9200:9200 \
    -p 9300:9300 \
    -e "discovery.type=single-node" \
    -e "xpack.security.enabled=false" \
    docker.elastic.co/elasticsearch/elasticsearch:8.4.3
```

**Answer**

> "build_hash" : "42f05b9372a9a4a470db3b52817899b99a76ee73"

In [1]:
!curl localhost:9200

{
  "name" : "4e546bc2ef36",
  "cluster_name" : "docker-cluster",
  "cluster_uuid" : "E_btB9GnRv6eKVW1bc5bAw",
  "version" : {
    "number" : "8.4.3",
    "build_flavor" : "default",
    "build_type" : "docker",
    "build_hash" : "42f05b9372a9a4a470db3b52817899b99a76ee73",
    "build_date" : "2022-10-04T07:17:24.662462378Z",
    "build_snapshot" : false,
    "lucene_version" : "9.3.0",
    "minimum_wire_compatibility_version" : "7.17.0",
    "minimum_index_compatibility_version" : "7.0.0"
  },
  "tagline" : "You Know, for Search"
}


## Getting the data

Now let's get the FAQ data. You can run this snippet:

In [2]:
import requests 

docs_url = 'https://github.com/DataTalksClub/llm-zoomcamp/blob/main/01-intro/documents.json?raw=1'
docs_response = requests.get(docs_url)
documents_raw = docs_response.json()

documents = []

for course in documents_raw:
    course_name = course['course']

    for doc in course['documents']:
        doc['course'] = course_name
        documents.append(doc)

In [3]:
documents[0]

{'text': "The purpose of this document is to capture frequently asked technical questions\nThe exact day and hour of the course will be 15th Jan 2024 at 17h00. The course will start with the first  “Office Hours'' live.1\nSubscribe to course public Google Calendar (it works from Desktop only).\nRegister before the course starts using this link.\nJoin the course Telegram channel with announcements.\nDon’t forget to register in DataTalks.Club's Slack and join the channel.",
 'section': 'General course-related questions',
 'question': 'Course - When will the course start?',
 'course': 'data-engineering-zoomcamp'}

## Q2. Indexing the data

Index the data in the same way as was shown in the course videos. Make the `course` field a keyword and the rest should be text. 

Don't forget to install the ElasticSearch client for Python:

```bash
pip install elasticsearch
```

Which function do you use for adding your data to elastic?

* `insert`
* `index`
* `put`
* `add`

**Answer**

*index*

In [4]:
from elasticsearch import Elasticsearch
from tqdm.auto import tqdm

/home/ubuntu/miniconda3/envs/llm/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [5]:
es_client = Elasticsearch('http://localhost:9200') 

In [6]:
es_client.info

<bound method Elasticsearch.info of <Elasticsearch(['http://localhost:9200'])>>

In [7]:
index_settings = {
    "settings": {
        "number_of_shards": 1,
        "number_of_replicas": 0
    },
    "mappings": {
        "properties": {
            "text": {"type": "text"},
            "section": {"type": "text"},
            "question": {"type": "text"},
            "course": {"type": "keyword"} 
        }
    }
}

index_name = "course-questions"

# Create the index
if not es_client.indices.exists(index=index_name):
    es_client.indices.create(index=index_name, body=index_settings)

In [8]:
for doc in tqdm(documents):
    es_client.index(index=index_name, document=doc)

100%|██████████| 948/948 [00:05<00:00, 163.26it/s]


## Q3. Searching

Now let's search in our index. 

We will execute a query "How do I execute a command in a running docker container?". 

Use only `question` and `text` fields and give `question` a boost of 4, and use `"type": "best_fields"`.

What's the score for the top ranking result?

* 94.05
* 84.05
* 74.05
* 64.05

Look at the `_score` field.

In [9]:
query = 'How do I execute a command in a running docker container?'

In [10]:
search_query = {
    "size": 2,
    "query": {
        "bool": {
            "must": {
                "multi_match": {
                    "query": query,
                    "fields": ["question^4", "text"],
                    "type": "best_fields"
                }
            },
            # "filter": {
            #     "term": {
            #         "course": "machine-learning-zoomcamp"
            #     }
            # }
        }
    }
}

# Search for documents
response = es_client.search(index=index_name, body=search_query)

In [11]:
print(response)

{'took': 37, 'timed_out': False, '_shards': {'total': 1, 'successful': 1, 'skipped': 0, 'failed': 0}, 'hits': {'total': {'value': 865, 'relation': 'eq'}, 'max_score': 84.050095, 'hits': [{'_index': 'course-questions', '_id': 'sGItbJABEprz7rupO4IA', '_score': 84.050095, '_source': {'text': 'Launch the container image in interactive mode and overriding the entrypoint, so that it starts a bash command.\ndocker run -it --entrypoint bash <image>\nIf the container is already running, execute a command in the specific container:\ndocker ps (find the container-id)\ndocker exec -it <container-id> bash\n(Marcos MJD)', 'section': '5. Deploying Machine Learning Models', 'question': 'How do I debug a docker container?', 'course': 'machine-learning-zoomcamp'}}, {'_index': 'course-questions', '_id': 'vmItbJABEprz7rupLoCu', '_score': 75.54128, '_source': {'text': 'In case running pgcli  locally causes issues or you do not want to install it locally you can use it running in a Docker container instead.

In [12]:
top_score = response["hits"]["hits"][0]["_score"]

print(f"Top ranking result score: {top_score}")

Top ranking result score: 84.050095


## Q4. Filtering

Now let's only limit the questions to `machine-learning-zoomcamp`.

Return 3 results. What's the 3rd question returned by the search engine?

* How do I debug a docker container?
* How do I copy files from a different folder into docker container’s working directory?
* How do Lambda container images work?
* How can I annotate a graph?

In [13]:
search_query = {
    "size": 3,
    "query": {
        "bool": {
            "must": {
                "multi_match": {
                    "query": query,
                    "fields": ["question^4", "text"],
                    "type": "best_fields"
                }
            },
            "filter": {
                "term": {
                    "course": "machine-learning-zoomcamp"
                }
            }
        }
    }
}

# Search for documents
response = es_client.search(index=index_name, body=search_query)

In [14]:
print(response)

{'took': 26, 'timed_out': False, '_shards': {'total': 1, 'successful': 1, 'skipped': 0, 'failed': 0}, 'hits': {'total': {'value': 345, 'relation': 'eq'}, 'max_score': 84.050095, 'hits': [{'_index': 'course-questions', '_id': 'sGItbJABEprz7rupO4IA', '_score': 84.050095, '_source': {'text': 'Launch the container image in interactive mode and overriding the entrypoint, so that it starts a bash command.\ndocker run -it --entrypoint bash <image>\nIf the container is already running, execute a command in the specific container:\ndocker ps (find the container-id)\ndocker exec -it <container-id> bash\n(Marcos MJD)', 'section': '5. Deploying Machine Learning Models', 'question': 'How do I debug a docker container?', 'course': 'machine-learning-zoomcamp'}}, {'_index': 'course-questions', '_id': 'z2ItbJABEprz7rupO4Kt', '_score': 51.04628, '_source': {'text': "You can copy files from your local machine into a Docker container using the docker cp command. Here's how to do it:\nTo copy a file or dir

In [15]:
third_question = response["hits"]["hits"][2]["_source"]["question"]

print(f"Third Question: {third_question}")

Third Question: How do I copy files from a different folder into docker container’s working directory?


## Q5. Building a prompt

Now we're ready to build a prompt to send to an LLM. 

Take the records returned from Elasticsearch in Q4 and use this template to build the context. Separate context entries by two linebreaks (`\n\n`)
```python
context_template = """
Q: {question}
A: {text}
""".strip()
```

Now use the context you just created along with the "How do I execute a command in a running docker container?" question 
to construct a prompt using the template below:

```
prompt_template = """
You're a course teaching assistant. Answer the QUESTION based on the CONTEXT from the FAQ database.
Use only the facts from the CONTEXT when answering the QUESTION.

QUESTION: {question}

CONTEXT:
{context}
""".strip()
```

What's the length of the resulting prompt? (use the `len` function)

* 962
* 1462
* 1962
* 2462

In [16]:
prompt_template = """
You're a course teaching assistant. Answer the QUESTION based on the CONTEXT from the FAQ database.
Use only the facts from the CONTEXT when answering the QUESTION.

QUESTION: {question}

CONTEXT:
{context}
"""

context_template = """
Q: {question}
A: {text}
""".strip()

context = ""

for doc in response['hits']['hits']:
    context_entry = context_template.format(
        question=doc['_source']['question'],
        text=doc['_source']['text']
    )
    context += context_entry + "\n\n"

context = context.strip()

prompt = prompt_template.format(question=query, context=context).strip()

print(prompt)

You're a course teaching assistant. Answer the QUESTION based on the CONTEXT from the FAQ database.
Use only the facts from the CONTEXT when answering the QUESTION.

QUESTION: How do I execute a command in a running docker container?

CONTEXT:
Q: How do I debug a docker container?
A: Launch the container image in interactive mode and overriding the entrypoint, so that it starts a bash command.
docker run -it --entrypoint bash <image>
If the container is already running, execute a command in the specific container:
docker ps (find the container-id)
docker exec -it <container-id> bash
(Marcos MJD)

Q: How do I copy files from my local machine to docker container?
A: You can copy files from your local machine into a Docker container using the docker cp command. Here's how to do it:
To copy a file or directory from your local machine into a running Docker container, you can use the `docker cp command`. The basic syntax is as follows:
docker cp /path/to/local/file_or_directory container_id:

In [17]:
len(prompt)

1462

## Q6. Tokens

When we use the OpenAI Platform, we're charged by the number of 
tokens we send in our prompt and receive in the response.

The OpenAI python package uses `tiktoken` for tokenization:

```bash
pip install tiktoken
```

Let's calculate the number of tokens in our query: 

```python
encoding = tiktoken.encoding_for_model("gpt-4o")
```

Use the `encode` function. How many tokens does our prompt have?

* 122
* 222
* 322
* 422

Note: to decode back a token into a word, you can use the `decode_single_token_bytes` function:

```python
encoding.decode_single_token_bytes(63842)
```

In [18]:
import tiktoken

In [19]:
encoding = tiktoken.encoding_for_model("gpt-4o")
tokens = encoding.encode(prompt)

In [20]:
num_tokens = len(tokens)
print(num_tokens)

322


## Bonus: generating the answer (ungraded)

Let's send the prompt to OpenAI. What's the response?  

Note: you can replace OpenAI with Ollama. See module 2.

In [21]:
import os

from groq import Groq

client = Groq(
    api_key=os.environ.get("GROQ_API_KEY"),
)

chat_completion = client.chat.completions.create(
    messages=[
        {
            "role": "user",
            "content": prompt,
        }
    ],
    model="llama3-8b-8192",
)

response = chat_completion.choices[0].message.content

print(response)

According to the context, you can execute a command in a running Docker container by finding the container-id using `docker ps`, and then executing the command using `docker exec -it <container-id> bash`.


## Bonus: calculating the costs (ungraded)

Suppose that on average per request we send 150 tokens and receive back 250 tokens.

How much will it cost to run 1000 requests?

You can see the prices [here](https://openai.com/api/pricing/)

On June 17, the prices for gpt4o are:

* Input: $0.005 / 1K tokens
* Output: $0.015 / 1K tokens

You can redo the calculations with the values you got in Q6 and Q7.


**Calculation for 1000 requests**

In [22]:
input_tokens_per_request = 150
output_tokens_per_request = 250
requests = 1000

price_per_1k_input_tokens = 0.005
price_per_1k_output_tokens = 0.015

total_input_tokens = input_tokens_per_request * requests
total_output_tokens = output_tokens_per_request * requests

input_cost = (total_input_tokens/1000) * price_per_1k_input_tokens
output_cost = (total_output_tokens/1000) * price_per_1k_output_tokens

total_cost = input_cost + output_cost

print(f"Total cost for 1000 requests: ${total_cost:.2f}")

Total cost for 1000 requests: $4.50


**Calculations Q6 and Q7 values**

In [23]:
tokens = encoding.encode(response)
num_out_tokens = len(tokens)
print(num_out_tokens)

42


In [24]:
input_tokens_per_request = 322
output_tokens_per_request = 42
requests = 1000

price_per_1k_input_tokens = 0.005
price_per_1k_output_tokens = 0.015

total_input_tokens = input_tokens_per_request * requests
total_output_tokens = output_tokens_per_request * requests

input_cost = (total_input_tokens/1000) * price_per_1k_input_tokens
output_cost = (total_output_tokens/1000) * price_per_1k_output_tokens

total_cost = input_cost + output_cost

print(f"Total cost for 1000 requests using Q6 and Q7 values: ${total_cost:.2f}")

Total cost for 1000 requests using Q6 and Q7 values: $2.24
